In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import re
from urllib.request import urlopen
import json
from pandas.io.json import json_normalize
import pandas as pd, numpy as np

In [3]:
### Scraping Instagram for urls, likes, and  via Selenium ###
username='manofthekinfolk'
account='natgeo'
browser = webdriver.Chrome(executable_path='C:\\Users\\david\\Downloads\\chromedriver\\chromedriver.exe')
browser.get('https://www.instagram.com/'+account+'/?hl=en')
Pagelength = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# 1. Login (Burner Account
# 2. Scroll down for several minutes
# 3. Run code below

In [29]:
number_of_images = 10000
browser = webdriver.Chrome(executable_path='C:\\Users\\david\\Downloads\\chromedriver\\chromedriver.exe')
browser.get('https://www.instagram.com/'+account+'/?hl=en')
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
links=[]
while len(links) < number_of_images:
    source = browser.page_source
    data=bs(source, 'html.parser')
    body = data.find('body')
    script = body.find('span')
    for link in script.findAll('a'):
         if re.match("/p", link.get('href')):
            pre_link = links.append('https://www.instagram.com'+link.get('href'))
            if pre_link not in links:
                links.append(pre_link)
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/4.0);")
        time.sleep(5)
        new_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if last_height == new_height:
            break
        last_height = new_height
#     return links
        


In [30]:
len(links)

10021

In [31]:
result=pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data=bs(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data=json.loads(raw)
        posts =json_data['entry_data']['PostPage'][0]['graphql']
        posts= json.dumps(posts)
        posts = json.loads(posts)
        x = pd.DataFrame.from_dict(json_normalize(posts), orient='columns') 
        x.columns =  x.columns.str.replace("shortcode_media.", "")
        result=result.append(x)
       
    except:
        np.nan
#Just check for the duplicates
result = result.drop_duplicates(subset = 'shortcode')
result.index = range(len(result.index))

In [32]:
result2 = pd.concat([result.loc[:,'display_url'],result.loc[:,'edge_media_to_caption.edges'],result.loc[:,'edge_media_to_parent_comment.count'],result.loc[:,'edge_media_preview_like.count']], axis='columns')
result2 = result2.rename(columns={"display_url":"URL","edge_media_to_caption.edges":"UnfinishedCaption","edge_media_to_parent_comment.count":"Comment Count","edge_media_preview_like.count":"Like Count"})
result2['UnfinishedCaption']=result2['UnfinishedCaption'].astype(str)
result2.loc[:, 'Caption'] = result2.UnfinishedCaption.map(lambda x: x[20:-4])
result2 = pd.concat([result2.loc[:,'URL'],result2.loc[:,'Caption'],result2.loc[:,'Like Count'],result2.loc[:,'Comment Count']], axis='columns')
word = 'Video'   
result3 = result2[~result2["Caption"].str.contains(word, na=False)]
word = 'Photos'
result4 = result3[~result3["Caption"].str.contains(word, na=False)]
result4

,URL,Caption,Like Count,Comment Count
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
6,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nichole Sobecki @nicholesobecki | Omm...,114367,297
7,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
...,...,...,...,...
1087,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
1092,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
1093,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
1095,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


In [33]:
export_csv = result4.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Instagram_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


In [35]:
unedited_csv=result.to_csv (r'C:\Users\david\Pictures\UGCA\Final\Unedited_Instagram_dataframe.csv', index = None, header=True)

In [36]:
result4.index = range(len(result4.index))
result4

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nichole Sobecki @nicholesobecki | Omm...,114367,297
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
...,...,...,...,...
952,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
953,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
954,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
955,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


### Importing data from csv to Dataframe

In [47]:
df = pd.read_csv(r'C:\Users\david\Pictures\UGCA\Final\Instagram_dataframe.csv')
df

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Nichole Sobecki @nicholesobecki | Omm...,114367,297
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
...,...,...,...,...
952,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
953,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
954,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
955,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


### Test data set 

In [61]:
# test = df.loc[0:4,:]
full = df
full = pd.concat([df.loc[0:2,:],df.loc[4:188,:],df.loc[270:,:]])
#full = pd.concat([df.loc[4:231,:],df.loc[233:237,:],df.loc[239:,:]])
full.index = range(len(full.index))
full

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @gerdludwig | Known for their elabora...,149658,394
...,...,...,...,...
870,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Diana Markosian @markosian | A mother...,248822,604
871,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Keith Ladzinski @ladzinski | An engin...,229292,652
872,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Michael Yamashita @yamashitaphoto | A...,558047,1748
873,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Joel Sartore @joelsartore | This red-...,442934,1451


In [62]:
test = api_df.loc[0:20,:]
test

,URL,Caption,Like Count,Comment Count


### Actual Dataset 

In [64]:
api_df = full.loc[0:499,:]
api_df

,URL,Caption,Like Count,Comment Count
0,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @brianskerry | The U.S. Civil War sub...,84699,268
1,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Albert Allard @williamalberta...,168260,519
2,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by David Guttenfelder @dguttenfelder | T...,252696,491
3,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @lucasfogliaphoto | Ashley Klein does...,287042,1717
4,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by @gerdludwig | Known for their elabora...,149658,394
...,...,...,...,...
495,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Pete Muller @pete_k_muller | During t...,154168,329
496,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Adam Ferguson @adamfergusonstudio | I...,269581,643
497,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by William Daniels @williamodaniels | Ru...,296273,1272
498,https://instagram.fftw1-1.fna.fbcdn.net/v/t51....,Photo by Cory Richards @coryrichards | When tr...,446163,1157


# Google Vision API


In [65]:
list_of_urls = api_df["URL"].tolist() #change test to api_df when ready

In [ ]:
from google.cloud import vision
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "keyFile.json"

image_urls = list_of_urls

ls_of_label_for_url = []
for url in image_urls:
    image_uri = url
    label_list = []
    client = vision.ImageAnnotatorClient()
    image = vision.types.Image()
    image.source.image_uri = image_uri
    response = client.label_detection(image=image)
    time.sleep(1)
    print('Labels (and confidence score):')
    print('=' * 79)
    
    for label in response.label_annotations:
        label_list.append(label.description)
        print(f'{label.description} ({label.score*100.:.2f}%)')
#         time.sleep(1)
    ls_of_label_for_url.append(label_list)
    
print(ls_of_label_for_url)
print('Length of unedited list',len(ls_of_label_for_url))
ls_of_label_for_url = list(filter(None, ls_of_label_for_url))
print(ls_of_label_for_url)
print('Length of edited list', len(ls_of_label_for_url))

Labels (and confidence score):
Rays and skates (88.14%)
Fish (82.69%)
Underwater (81.39%)
Water (80.73%)
Manta ray (77.54%)
Organism (76.72%)
Marine biology (73.21%)
Stingray (71.06%)
Cartilaginous fish (61.89%)
Skate (60.42%)
Labels (and confidence score):
Photography (62.42%)
Labels (and confidence score):
Badlands (98.23%)
Mountainous landforms (98.08%)
Sky (97.49%)
Nature (95.35%)
Mountain (93.90%)
Rock (93.53%)
Formation (91.75%)
Cloud (91.28%)
Wilderness (89.94%)
Geology (88.88%)
Labels (and confidence score):
Rock (69.81%)
Tourism (69.71%)
Landscape (59.31%)
National park (54.88%)
Labels (and confidence score):
People (95.68%)
Community (84.84%)
Yellow (83.63%)
Tradition (76.61%)
Event (76.34%)
Temple (72.84%)
Tourism (68.09%)
Adaptation (66.97%)
Sari (66.58%)
Smile (64.11%)
Labels (and confidence score):
Water (95.78%)
Photograph (95.59%)
Sky (94.83%)
Sea (93.78%)
Skin (90.60%)
Beauty (90.38%)
Ocean (87.65%)
Vacation (86.62%)
Summer (83.16%)
Cloud (80.34%)
Labels (and confidenc

Labels (and confidence score):
Design (73.01%)
Textile (68.45%)
Colorfulness (67.83%)
Pattern (65.68%)
Photography (62.42%)
Crowd (50.70%)
Labels (and confidence score):
Hair (98.69%)
Face (97.95%)
Beauty (92.50%)
Lip (90.57%)
Lady (89.53%)
Hairstyle (89.53%)
Smile (88.74%)
Eyebrow (87.64%)
Brown hair (85.71%)
Chin (85.04%)
Labels (and confidence score):
Mammal (98.90%)
Hyena (98.78%)
Vertebrate (98.51%)
Wildlife (97.71%)
Terrestrial animal (96.69%)
Spotted hyena (94.62%)
Carnivore (93.42%)
Snout (73.43%)
Savanna (56.52%)
Felidae (54.08%)
Labels (and confidence score):
Sky (97.19%)
Nature (95.35%)
Horizon (94.92%)
Atmosphere (94.29%)
Atmospheric phenomenon (94.22%)
Freezing (91.54%)
Light (89.30%)
Winter (88.94%)
Morning (88.02%)
Evening (87.49%)
Labels (and confidence score):
Night (53.94%)
Labels (and confidence score):
Sky (97.45%)
Water (95.13%)
Blue (92.70%)
Sea (85.70%)
Atmosphere (84.04%)
Cloud (83.81%)
Ocean (83.12%)
Horizon (70.93%)
Rock (69.81%)
Tree (68.69%)
Labels (and conf

Labels (and confidence score):
People (96.00%)
Human (78.89%)
Adaptation (74.08%)
Smile (64.11%)
Temple (63.72%)
Event (62.74%)
Child (61.35%)
Art (58.13%)
Crowd (57.81%)
Flesh (53.55%)
Labels (and confidence score):
Green (97.68%)
Sky (88.58%)
Aurora (80.47%)
Grass (68.80%)
Tree (68.69%)
Landscape (68.08%)
Grassland (57.75%)
Night (57.54%)
House (53.69%)
Labels (and confidence score):
Blue (97.16%)
Fish (87.12%)
Fish (81.99%)
Organism (81.17%)
Water (80.73%)
Marine biology (77.85%)
Electric blue (58.19%)
Hairfinned silverfish (54.38%)
Deep sea fish (53.86%)
Turquoise (51.21%)
Labels (and confidence score):
Youth (82.32%)
Community (81.46%)
Event (69.43%)
Adaptation (66.97%)
Leisure (56.19%)
Labels (and confidence score):
Furniture (87.46%)
Room (80.06%)
Bed (71.91%)
Bedroom (67.48%)
Textile (65.63%)
Feather (60.27%)
Bed sheet (55.40%)
World (53.14%)
Labels (and confidence score):
Wildlife (97.06%)
Hyena (90.16%)
Grassland (88.67%)
Natural landscape (88.55%)
Savanna (88.54%)
Spotted hy

Labels (and confidence score):
Table (90.91%)
Room (86.48%)
Furniture (78.62%)
Interior design (77.19%)
Restaurant (75.80%)
Vacation (75.20%)
Leisure (70.92%)
Coffee table (67.34%)
House (59.29%)
Wallpaper (54.56%)
Labels (and confidence score):
Mountainous landforms (99.02%)
Mountain (98.36%)
Mountain range (93.41%)
Ridge (90.62%)
Sky (88.58%)
Summit (84.65%)
Geological phenomenon (79.50%)
Massif (76.36%)
Fell (75.14%)
Highland (74.05%)
Labels (and confidence score):
Hair (98.84%)
Face (98.54%)
Facial expression (94.44%)
Chin (92.41%)
Hairstyle (91.19%)
Head (90.14%)
Eyebrow (89.94%)
Nose (86.99%)
Beauty (86.39%)
Blond (84.73%)
Labels (and confidence score):
Architecture (95.10%)
Sky (94.21%)
Reflection (89.89%)
Building (83.93%)
Mixed-use (83.41%)
Night (78.51%)
Urban area (74.97%)
Facade (72.49%)
House (71.72%)
Real estate (67.23%)
Labels (and confidence score):
Metropolitan area (97.85%)
Cityscape (97.50%)
Urban area (97.35%)
Metropolis (97.25%)
City (96.79%)
Night (92.62%)
Human s

Labels (and confidence score):
Dress (95.08%)
Pink (94.59%)
Lady (93.12%)
Clothing (92.83%)
Beauty (89.61%)
Gown (88.62%)
Skin (88.11%)
Shoulder (87.33%)
Bride (83.73%)
Wedding dress (82.87%)
Labels (and confidence score):
Sky (98.65%)
Blue (91.18%)
Cloud (82.83%)
Horizon (80.53%)
Silhouette (77.29%)
Sea (74.93%)
Atmosphere (72.90%)
Calm (71.15%)
Astronomical object (70.91%)
Evening (70.39%)
Labels (and confidence score):
Aurora (98.85%)
Sky (98.23%)
Nature (97.20%)
Green (97.05%)
Atmospheric phenomenon (91.25%)
Natural landscape (88.55%)
Atmosphere (88.41%)
Geological phenomenon (84.87%)
Landscape (74.41%)
Night (72.85%)
Labels (and confidence score):
Parka (87.87%)
Winter (87.08%)
Outerwear (79.78%)
Snow (79.77%)
Coat (78.51%)
Fur (69.13%)
Headgear (67.48%)
Jacket (64.51%)
Photography (62.42%)
Shotgun (56.83%)
Labels (and confidence score):
Darkness (88.70%)
Lady (86.41%)
Smoke (78.68%)
Human (74.68%)
Human body (71.52%)
Hand (66.00%)
Performance art (60.58%)
Flesh (57.73%)
Portrait 

...

Labels (and confidence score):
Wildlife (96.82%)
Terrestrial animal (89.32%)
Jungle (83.60%)
Organism (79.41%)
Tree (74.70%)
Adaptation (74.08%)
Grass (68.80%)
Night (66.64%)
Plant (66.16%)
Forest (65.85%)
Labels (and confidence score):
Mountainous landforms (99.32%)
Mountain (99.00%)
Highland (98.36%)
Sky (97.91%)
Mountain range (96.84%)
Cloud (95.46%)
Hill station (88.64%)
Natural landscape (88.55%)
Atmospheric phenomenon (87.48%)
Hill (87.25%)
Labels (and confidence score):
Ice (94.72%)
Freezing (91.89%)
Glacier (90.30%)
Glacial landform (89.66%)
Polar ice cap (86.12%)
Ice cap (85.45%)
Formation (78.98%)
Winter (78.86%)
Iceberg (78.38%)
Mountain (77.51%)
Labels (and confidence score):
Yellow (86.49%)
Event (72.28%)
Tradition (66.04%)
Temple (63.72%)
Photography (62.42%)
Sari (57.83%)
Festival (54.86%)
Performance (50.90%)
Ceremony (50.85%)
Labels (and confidence score):
Sky (93.82%)
Silhouette (89.89%)
Standing (81.92%)
Cloud (81.69%)
Water (80.73%)
Photography (72.45%)
Backlighting

Labels (and confidence score):
Building (75.83%)
Temple (73.41%)
Architecture (72.63%)
Street (70.50%)
Door (69.33%)
Tourism (56.89%)
City (56.19%)
Pedestrian (55.47%)
Hindu temple (55.16%)
Vacation (54.60%)
Labels (and confidence score):
Sky (97.06%)
Atmospheric phenomenon (94.76%)
Light (92.81%)
Atmosphere (91.68%)
Darkness (87.00%)
Lighting (84.01%)
Cloud (82.83%)
Night (79.02%)
Sunlight (77.38%)
Space (68.84%)
Labels (and confidence score):
Vertebrate (98.51%)
Mammal (97.06%)
Facial expression (94.48%)
Wildlife (94.01%)
Yawn (83.61%)
Snout (78.40%)
Terrestrial animal (74.44%)
Carnivore (71.84%)
Grass (68.80%)
Photography (62.42%)
Labels (and confidence score):
Sky (98.96%)
Nature (94.94%)
Star (90.01%)
Night (89.43%)
Cloud (82.83%)
Atmosphere (82.49%)
Astronomical object (80.49%)
Mountain (75.28%)
National park (73.97%)
Landscape (72.70%)
Labels (and confidence score):
Sitting (89.76%)
Human (74.68%)
Adaptation (66.97%)
Mother (53.35%)
Child (52.03%)
Labels (and confidence score):


Labels (and confidence score):
People (93.70%)
Male (80.90%)
Human (76.88%)
Adaptation (74.08%)
Smile (72.92%)
Cool (70.32%)
Photography (62.42%)
Happy (54.13%)
World (53.14%)
Labels (and confidence score):
Transport (93.95%)
Hill station (91.83%)
Track (88.70%)
Atmospheric phenomenon (85.69%)
Sky (73.26%)
Travel (69.35%)
Vehicle (63.74%)
Mountain (63.04%)
Thoroughfare (57.66%)
Railway (57.26%)
Labels (and confidence score):
Mountainous landforms (94.33%)
Natural environment (87.15%)
Wadi (86.13%)
Geological phenomenon (84.87%)
Sky (82.21%)
Mountain (81.64%)
Geology (80.95%)
Sand (79.45%)
Human (75.91%)
Landscape (74.41%)
Labels (and confidence score):
Seaweed (95.53%)
Kelp (94.62%)
Macrocystis pyrifera (93.43%)
Macrocystis (90.04%)
Underwater (89.40%)
Water (80.73%)
Organism (79.41%)
Marine biology (75.67%)
Algae (73.46%)
Aquatic plant (66.13%)
Labels (and confidence score):
Sky (97.00%)
Water (95.13%)
Horizon (93.85%)
Sea (89.57%)
Cloud (87.76%)
Ocean (84.07%)
Boat (83.58%)
Vehicle (

Labels (and confidence score):
Water (91.00%)
Sea (90.55%)
Wave (90.25%)
Wind wave (89.58%)
Ocean (87.12%)
Whale (81.47%)
Humpback whale (75.41%)
Marine mammal (74.10%)
Tide (67.58%)
grey whale (65.27%)
Labels (and confidence score):
People (93.43%)
Adaptation (83.28%)
Human (81.16%)
Photography (62.42%)
Tribe (58.15%)
Statue (51.32%)
Art (50.22%)
Labels (and confidence score):
Formation (97.31%)
Rock (93.32%)
Cave (90.07%)
Geology (82.91%)
Erosion (69.48%)
Geological phenomenon (61.63%)
Lava cave (61.28%)
World (60.53%)
Landscape (59.31%)
Coastal and oceanic landforms (56.92%)
Labels (and confidence score):
Mountainous landforms (95.80%)
Fur seal (95.40%)
Mountain (94.75%)
Mountain range (87.57%)
Wilderness (85.57%)
Seal (80.72%)
Alps (74.18%)
Snow (69.76%)
Wildlife (67.62%)
National park (67.41%)
Labels (and confidence score):
Branch (97.09%)
Tree (91.39%)
Silhouette (89.02%)
Sky (88.58%)
Wildlife (83.13%)
Twig (72.22%)
Plant (62.44%)
Shadow (59.29%)
Trunk (58.89%)
Labels (and confid

Labels (and confidence score):
Circle (72.88%)
Space (66.89%)
Photography (62.42%)
Flash photography (58.31%)
Artist (53.39%)
World (53.14%)
Art (50.22%)
Labels (and confidence score):
Black (95.72%)
White (95.47%)
Black-and-white (92.00%)
Monochrome (89.03%)
Monochrome photography (86.25%)
Snapshot (82.03%)
Street (76.35%)
Sitting (73.72%)
Photography (67.79%)
Style (55.52%)
Labels (and confidence score):
Marine biology (97.08%)
Marine mammal (96.16%)
Underwater (95.26%)
Water (93.89%)
Dolphin (93.16%)
Blue (92.70%)
Fin (92.26%)
Cetacea (91.51%)
Spinner dolphin (87.10%)
Common bottlenose dolphin (85.14%)
Labels (and confidence score):
Tortoise (99.28%)
Vertebrate (98.51%)
Galápagos tortoise (97.20%)
Turtle (96.93%)
Reptile (96.15%)
Chelonoidis (95.03%)
Pond turtle (89.15%)
Animal figure (82.55%)
Common snapping turtle (75.57%)
Terrestrial animal (74.44%)
Labels (and confidence score):
Mammal (98.90%)
Vertebrate (98.51%)
Wildlife (97.43%)
Canidae (96.86%)
Canis (93.66%)
Wolf (90.21%)
C